# Paths of valid images

In this notebook, we extract paths of images for which the solar radiation values are valid.

In [5]:
import numpy as np
import pandas as pd
from fnmatch import fnmatch
import os

### for the ~16000 valid images, 5 min spaced.

In [6]:
camera_name = 'mobotix1'
images_folder_path = "../../Solais_Data/FTP"

In [40]:
GHI_target = pd.read_csv("../Models/data/dataset_final.csv")
GHI_target.rename(columns={"Unnamed: 0": "Timestamp"}, inplace=True)
GHI_target.head(10)

,Timestamp,gray_concentric0_mean,gray_concentric0_std,gray_concentric0_smoothness,gray_concentric0_skewness,gray_concentric0_entropy,gray_concentric1_mean,gray_concentric1_std,gray_concentric1_smoothness,gray_concentric1_skewness,...,clearsky_dni,clearsky_dhi,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time,ghi_clipped,Kc
0,2020-06-20 06:33:00,0.033073,0.003901,0.000015,0.076168,3.002126,0.040017,0.006983,0.000049,1.063612,...,523.008808,0.559849,87.751066,88.030116,2.248934,1.969884,66.387850,-1.636136,13.3675,0.633730
1,2020-06-20 06:38:00,0.042338,0.004321,0.000019,0.292108,3.101799,0.051748,0.009400,0.000088,1.225713,...,619.571102,0.974757,86.693645,86.913528,3.306355,3.086472,66.649220,-1.636898,18.8975,0.514661
2,2020-06-20 06:43:00,0.026753,0.002712,0.000007,0.880031,1.859379,0.033751,0.006900,0.000048,1.214367,...,694.243703,1.447908,85.615134,85.794776,4.384866,4.205224,66.901419,-1.637659,25.2800,0.463540
3,2020-06-20 06:48:00,0.022493,0.002691,0.000007,1.104797,1.419124,0.028993,0.007165,0.000051,1.216869,...,752.065994,1.958091,84.523011,84.673958,5.476989,5.326042,67.144588,-1.638421,31.8800,0.432273
4,2020-06-20 06:53:00,0.024859,0.002889,0.000008,0.267025,2.387260,0.030767,0.008043,0.000065,1.532496,...,797.436717,2.492088,83.421527,83.551169,6.578473,6.448831,67.378853,-1.639183,40.9900,0.436716
5,2020-06-20 06:58:00,0.029551,0.003826,0.000015,0.335292,2.150311,0.037085,0.009809,0.000096,1.450653,...,833.635163,3.041682,82.313216,82.426501,7.686784,7.573499,67.604326,-1.639944,50.0675,0.437054
6,2020-06-20 07:03:00,0.033474,0.004343,0.000019,0.423644,2.345576,0.043297,0.013431,0.000180,1.734723,...,863.002859,3.601629,81.199667,81.300042,8.800333,8.699958,67.821102,-1.640706,57.1100,0.421029
7,2020-06-20 07:08:00,0.036386,0.005623,0.000032,1.135848,2.990002,0.049542,0.021758,0.000473,1.735035,...,887.202666,4.168459,80.081931,80.171878,9.918069,9.828122,68.029263,-1.641467,63.6350,0.405344
8,2020-06-20 07:13:00,0.045561,0.010903,0.000119,1.699767,3.803160,0.063757,0.029183,0.000851,1.276920,...,907.426127,4.739776,78.960732,79.042093,11.039268,10.957907,68.228874,-1.642229,77.7025,0.435295
9,2020-06-20 07:18:00,0.071576,0.021793,0.000475,0.743199,5.405326,0.088466,0.036290,0.001315,0.796047,...,924.539449,5.313862,77.836593,77.910766,12.163407,12.089234,68.419987,-1.642991,98.0925,0.490157


In [9]:
GHI_target["Timestamp"] = pd.to_datetime(GHI_target["Timestamp"], format='%Y-%m-%d %H:%M:%S')
GHI_target["date"] = GHI_target["Timestamp"].dt.date
GHI_target["time"] = GHI_target["Timestamp"].dt.time
GHI_target = GHI_target[["Timestamp", "date", "time"]]

In [5]:
GHI_target

,Timestamp,date,time
0,2020-06-20 06:33:00,2020-06-20,06:33:00
1,2020-06-20 06:38:00,2020-06-20,06:38:00
2,2020-06-20 06:43:00,2020-06-20,06:43:00
3,2020-06-20 06:48:00,2020-06-20,06:48:00
4,2020-06-20 06:53:00,2020-06-20,06:53:00
...,...,...,...
17365,2020-10-22 17:48:00,2020-10-22,17:48:00
17366,2020-10-22 17:53:00,2020-10-22,17:53:00
17367,2020-10-22 17:58:00,2020-10-22,17:58:00
17368,2020-10-22 18:03:00,2020-10-22,18:03:00


In [10]:
valid_days = GHI_target["date"].unique()
day0 = valid_days[0]

In [11]:
GHI_target[GHI_target.date == day0]

,Timestamp,date,time
0,2020-06-20 06:33:00,2020-06-20,06:33:00
1,2020-06-20 06:38:00,2020-06-20,06:38:00
2,2020-06-20 06:43:00,2020-06-20,06:43:00
3,2020-06-20 06:48:00,2020-06-20,06:48:00
4,2020-06-20 06:53:00,2020-06-20,06:53:00
...,...,...,...
143,2020-06-20 18:28:00,2020-06-20,18:28:00
144,2020-06-20 18:33:00,2020-06-20,18:33:00
145,2020-06-20 18:38:00,2020-06-20,18:38:00
146,2020-06-20 18:43:00,2020-06-20,18:43:00


In [12]:
def get_valid_patterns(day, dataset):
    time_gen = [str(time)[:-3] for time in dataset[dataset.date == day]["time"]]
    return [time[0:2]+"_"+time[3:5]+"*" for time in time_gen]

In [13]:
patterns0 = get_valid_patterns(day0, GHI_target)

In [14]:
def get_valid_paths(day, dataset):
    """ Gets path for every image of day (datetime.date)
    """    
    day_name = str(day)
    print(f"Compute paths for {day_name}")
    camera_day_path = images_folder_path + '/' + day_name + '/' + camera_name + '/'
    filenames = os.listdir(camera_day_path)
    return filenames
    valid_patterns = get_valid_patterns(day, dataset=dataset)
    # format is "HH:MM:"
    for pattern in valid_patterns:
        for filename in filenames:
            if fnmatch(filename, pattern):
                path = camera_day_path + filename
                yield path

In [15]:
filenames0 = get_valid_paths(day0, GHI_target)

In [16]:
for pattern in patterns0:
    for filename in filenames0:
        if fnmatch(filename, pattern):
            print("match")

Compute paths for 2020-06-20


In [17]:
def path_per_day(day, dataset):
    return (path for path in get_valid_paths(day, dataset))

In [18]:
path_gen = (path_per_day(day, GHI_target) for day in valid_days)

In [10]:
Text = open("../../Solais_Data/valid_img_paths.txt", "a+")
for daily_gen in path_gen:
    for path in daily_gen:
        Text.write("\n")
        Text.write(path)
Text.close()

Compute paths for 2020-06-20
Compute paths for 2020-06-21
Compute paths for 2020-06-22
Compute paths for 2020-06-23
Compute paths for 2020-06-24
Compute paths for 2020-06-25
Compute paths for 2020-06-26
Compute paths for 2020-06-27
Compute paths for 2020-06-28
Compute paths for 2020-06-29
Compute paths for 2020-06-30
Compute paths for 2020-07-01
Compute paths for 2020-07-02
Compute paths for 2020-07-03
Compute paths for 2020-07-04
Compute paths for 2020-07-05
Compute paths for 2020-07-06
Compute paths for 2020-07-07
Compute paths for 2020-07-08
Compute paths for 2020-07-09
Compute paths for 2020-07-10
Compute paths for 2020-07-11
Compute paths for 2020-07-12
Compute paths for 2020-07-13
Compute paths for 2020-07-14
Compute paths for 2020-07-15
Compute paths for 2020-07-16
Compute paths for 2020-07-17
Compute paths for 2020-07-18
Compute paths for 2020-07-19
Compute paths for 2020-07-20
Compute paths for 2020-07-21
Compute paths for 2020-07-22
Compute paths for 2020-07-23
Compute paths 